In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import torch
import open_clip
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from torchvision import transforms
from transformers import AutoModel
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
file_path = "/fs/ess/PAS2136/Rayeed/BritishCarabids/British Carabids/BritishCarabids.csv"
brit_df = pd.read_csv(file_path)
brit_df.rename(columns={"CanonicalName": "ScientificName"}, inplace=True)
cols = ['ImageFilePath', 'ScientificName']
df = brit_df[cols]
df = df.dropna()
df.head(2)

,ImageFilePath,ScientificName
0,/fs/ess/PAS2136/Rayeed/BritishCarabids/British...,Asaphidion curtum
1,/fs/ess/PAS2136/Rayeed/BritishCarabids/British...,Asaphidion curtum


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModel.from_pretrained("facebook/dinov2-base").to(device)


preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [5]:
def extract_features(image_path):
    
    image = Image.open(image_path).convert("RGB")
    
    image_tensor = preprocess(image).unsqueeze(0).to(device)

    with torch.no_grad():
        features = model(image_tensor).last_hidden_state.mean(dim=1)

    
    return features.cpu().numpy()
    

In [6]:
X = np.vstack([extract_features(img) for img in tqdm(df["ImageFilePath"], desc="Processing ...") ])

print(f"Extracted feature shape: {X.shape}")


le = LabelEncoder()

y = le.fit_transform(df["ScientificName"])

df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df_indices, test_size=0.2, random_state=42)

test_df = df.loc[test_idx].copy()


Processing ...: 100%|██████████| 63077/63077 [15:00<00:00, 70.08it/s] 


Extracted feature shape: (63077, 768)


In [7]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


In [8]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef


seed = 99


models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=5), 
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(256, 128), activation='logistic', max_iter=200, random_state=seed)
}

predictions = {}

metrics = {}


for name, model in models.items():

    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds

    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)

    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}

    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T


NaiveBayes                | Acc: 18.17% | Prec: 32.39% | Rec: 18.17% | F1: 18.15% | Bal Acc: 22.72% | MCC: 0.1798
LogisticRegression        | Acc: 81.06% | Prec: 81.55% | Rec: 81.06% | F1: 81.03% | Bal Acc: 78.48% | MCC: 0.8096
NearestNeighbor           | Acc: 32.36% | Prec: 40.59% | Rec: 32.36% | F1: 33.26% | Bal Acc: 29.85% | MCC: 0.3203
MLP_Baseline              | Acc: 78.07% | Prec: 78.38% | Rec: 78.07% | F1: 77.98% | Bal Acc: 75.53% | MCC: 0.7795


In [9]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,ScientificName,Pred_NaiveBayes,Pred_LogisticRegression,Pred_NearestNeighbor,Pred_MLP_Baseline
6930,/fs/ess/PAS2136/Rayeed/BritishCarabids/British...,Harpalus rubripes,Amara spreta,Harpalus rubripes,Anisodactylus binotatus,Amara apricaria
2549,/fs/ess/PAS2136/Rayeed/BritishCarabids/British...,Bembidion fumigatum,Ocys quinquestriatus,Bembidion assimile,Bembidion assimile,Bembidion lunulatum


In [13]:
test_df.to_csv("/users/PAS2136/rayees/CV4A - Benchmarking/BritishCarabids/7.DINOv2-linear-probing-species.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/CV4A - Benchmarking/BritishCarabids/7.DINOv2-linear-probing-species-metrics.csv", index=False)